In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('Aula de Spark 5').enableHiveSupport().getOrCreate()

In [3]:
spark

In [ ]:
#lendo do S3

In [4]:
results = spark.read.csv('s3a://camada-bronze-aula5/futebol', header = True)

In [5]:
results.show(5)

+----------+-------------+-------------+--------------+--------------+--------------+--------+--------------+-----------+
|      date|home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName|cityCity|countryCountry|neutralTRUE|
+----------+-------------+-------------+--------------+--------------+--------------+--------+--------------+-----------+
|1872-11-30|     Scotland|      England|             0|             0|      Friendly| Glasgow|      Scotland|      FALSE|
|1873-03-08|      England|     Scotland|             4|             2|      Friendly|  London|       England|      FALSE|
|1874-03-07|     Scotland|      England|             2|             1|      Friendly| Glasgow|      Scotland|      FALSE|
|1875-03-06|      England|     Scotland|             2|             2|      Friendly|  London|       England|      FALSE|
|1876-03-04|     Scotland|      England|             3|             0|      Friendly| Glasgow|      Scotland|      FALSE|
+----------+------------

In [6]:
results.describe().show()

+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+--------------+-----------+
|summary|      date|home_teamName|away_teamName|    home_scoreHome|    away_scoreAway| tournamentName|           cityCity|countryCountry|neutralTRUE|
+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+--------------+-----------+
|  count|     40839|        40839|        40839|             40839|             40839|          40839|              40839|         40839|      40839|
|   mean|      null|         null|         null|1.7457087587844953| 1.188104507945836|           null|               null|          null|       null|
| stddev|      null|         null|         null| 1.749144918253105|1.4051198000976632|           null|               null|          null|       null|
|    min|1872-11-30|     Abkhazia|     Abkhazia|                 0|                 0|ABCS Tournamen

In [7]:
results.printSchema()

root
 |-- date: string (nullable = true)
 |-- home_teamName: string (nullable = true)
 |-- away_teamName: string (nullable = true)
 |-- home_scoreHome: string (nullable = true)
 |-- away_scoreAway: string (nullable = true)
 |-- tournamentName: string (nullable = true)
 |-- cityCity: string (nullable = true)
 |-- countryCountry: string (nullable = true)
 |-- neutralTRUE: string (nullable = true)



In [ ]:
# renomeando e mundando tipo

In [8]:
partidas = (
    results
    .drop('neutralTrue')
    .withColumnRenamed('date','DATA')
    .withColumnRenamed('home_teamName','MANDANTE')
    .withColumnRenamed('away_teamName','VISITANTE')
    .withColumnRenamed('home_scoreHome','GOL_MANDANTE').withColumn('GOL_MANDANTE',col('GOL_MANDANTE').cast('int'))
    .withColumnRenamed('away_scoreAway','GOL_VISITANTE').withColumn('GOL_VISITANTE',col('GOL_VISITANTE').cast('int'))
    .withColumnRenamed('tournamentName','TORNEIO')
    .withColumnRenamed('cityCity','CIDADE')
    .withColumnRenamed('countryCountry','PAIS')
)

In [9]:
partidas.printSchema()

root
 |-- DATA: string (nullable = true)
 |-- MANDANTE: string (nullable = true)
 |-- VISITANTE: string (nullable = true)
 |-- GOL_MANDANTE: integer (nullable = true)
 |-- GOL_VISITANTE: integer (nullable = true)
 |-- TORNEIO: string (nullable = true)
 |-- CIDADE: string (nullable = true)
 |-- PAIS: string (nullable = true)



In [10]:
partidas_pandas = partidas.toPandas()

In [11]:
partidas_pandas

,DATA,MANDANTE,VISITANTE,GOL_MANDANTE,GOL_VISITANTE,TORNEIO,CIDADE,PAIS
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland
1,1873-03-08,England,Scotland,4,2,Friendly,London,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,London,England
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland
...,...,...,...,...,...,...,...,...
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India


# Python (Pyspark)
# SQL

In [ ]:
#convetendo para sql

In [12]:
partidas.createOrReplaceTempView('v_partidas')

In [13]:
spark.sql('select * from v_partidas limit 2').show()

+----------+--------+---------+------------+-------------+--------+-------+--------+
|      DATA|MANDANTE|VISITANTE|GOL_MANDANTE|GOL_VISITANTE| TORNEIO| CIDADE|    PAIS|
+----------+--------+---------+------------+-------------+--------+-------+--------+
|1872-11-30|Scotland|  England|           0|            0|Friendly|Glasgow|Scotland|
|1873-03-08| England| Scotland|           4|            2|Friendly| London| England|
+----------+--------+---------+------------+-------------+--------+-------+--------+



In [14]:
partidas.describe().show()

+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+-----------+
|summary|      DATA|     MANDANTE|    VISITANTE|      GOL_MANDANTE|     GOL_VISITANTE|        TORNEIO|             CIDADE|       PAIS|
+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+-----------+
|  count|     40839|        40839|        40839|             40839|             40839|          40839|              40839|      40839|
|   mean|      null|         null|         null|1.7457087587844953| 1.188104507945836|           null|               null|       null|
| stddev|      null|         null|         null| 1.749144918253105|1.4051198000976632|           null|               null|       null|
|    min|1872-11-30|     Abkhazia|     Abkhazia|                 0|                 0|ABCS Tournament|6th of October City|Afghanistan|
|    max|2019-07-20|Åland Islands|Åland Islands|       

In [26]:
(
    partidas \
        .select('TORNEIO') \
        .distinct() \
        .orderBy('TORNEIO') \
        .show(3,truncate=False)
)

+---------------------------+
|TORNEIO                    |
+---------------------------+
|ABCS Tournament            |
|AFC Asian Cup              |
|AFC Asian Cup qualification|
+---------------------------+
only showing top 3 rows



In [31]:
spark.sql('select distinct torneio from v_partidas order by 1').show(3,truncate=False)

+---------------------------+
|torneio                    |
+---------------------------+
|ABCS Tournament            |
|AFC Asian Cup              |
|AFC Asian Cup qualification|
+---------------------------+
only showing top 3 rows



In [43]:
spark.sql('SELECT TORNEIO, COUNT(DATA) AS total_partidas \
FROM v_partidas \
GROUP BY TORNEIO \
ORDER BY total_partidas desc;').show(10, truncate=False)

+------------------------------------+--------------+
|TORNEIO                             |total_partidas|
+------------------------------------+--------------+
|Friendly                            |16911         |
|FIFA World Cup qualification        |7100          |
|UEFA Euro qualification             |2432          |
|African Cup of Nations qualification|1616          |
|FIFA World Cup                      |900           |
|Copa América                        |813           |
|AFC Asian Cup qualification         |724           |
|African Cup of Nations              |690           |
|CECAFA Cup                          |620           |
|CFU Caribbean Cup qualification     |606           |
+------------------------------------+--------------+
only showing top 10 rows



In [42]:
(
    partidas
    .select('TORNEIO')
    .groupBy('TORNEIO')
    .count()
    .orderBy('count', ascending=False)
    .show(3,truncate=False)
)

+----------------------------+-----+
|TORNEIO                     |count|
+----------------------------+-----+
|Friendly                    |16911|
|FIFA World Cup qualification|7100 |
|UEFA Euro qualification     |2432 |
+----------------------------+-----+
only showing top 3 rows



In [63]:
df_fwc_brasil = (
    partidas
    .filter(partidas.TORNEIO == "FIFA World Cup")
    .filter((partidas.MANDANTE == "Brazil") | (partidas.VISITANTE == "Brazil"))
)

In [64]:
df_fwc_brasil.count()

109

In [69]:
df_fwc_brasil.describe()

DataFrame[summary: string, DATA: string, MANDANTE: string, VISITANTE: string, GOL_MANDANTE: string, GOL_VISITANTE: string, TORNEIO: string, CIDADE: string, PAIS: string]

In [65]:
#QUANTAS PARTIDAS BRASIL EMPATOU

In [73]:
df_fwc_brasil \
    .filter(df_fwc_brasil.GOL_MANDANTE == df_fwc_brasil.GOL_VISITANTE).count()

18

In [67]:
#QUANTAS PARTIDAS BRASIL GANHOU

In [84]:
(
df_fwc_brasil
    .filter(((df_fwc_brasil.GOL_MANDANTE > df_fwc_brasil.GOL_VISITANTE) & (df_fwc_brasil.MANDANTE == 'Brazil'))
    |
    ((df_fwc_brasil.GOL_MANDANTE < df_fwc_brasil.GOL_VISITANTE) & (df_fwc_brasil.VISITANTE == 'Brazil'))).count()
)

73

In [ ]:
#QUANTAS PARTIDAS BRASIL PERDEU

In [86]:
(
df_fwc_brasil
    .filter(((df_fwc_brasil.GOL_MANDANTE > df_fwc_brasil.GOL_VISITANTE) & (df_fwc_brasil.VISITANTE == 'Brazil'))
    |
    ((df_fwc_brasil.GOL_MANDANTE < df_fwc_brasil.GOL_VISITANTE) & (df_fwc_brasil.MANDANTE == 'Brazil'))).show()
)

+----------+-----------+-----------+------------+-------------+--------------+-----------------+------------+
|      DATA|   MANDANTE|  VISITANTE|GOL_MANDANTE|GOL_VISITANTE|       TORNEIO|           CIDADE|        PAIS|
+----------+-----------+-----------+------------+-------------+--------------+-----------------+------------+
|1930-07-14|     Brazil| Yugoslavia|           1|            2|FIFA World Cup|       Montevideo|     Uruguay|
|1934-05-27|     Brazil|      Spain|           1|            3|FIFA World Cup|            Genoa|       Italy|
|1938-06-16|     Brazil|      Italy|           1|            2|FIFA World Cup|        Marseille|      France|
|1950-07-16|     Brazil|    Uruguay|           1|            2|FIFA World Cup|   Rio de Janeiro|      Brazil|
|1954-06-27|     Brazil|    Hungary|           2|            4|FIFA World Cup|            Berne| Switzerland|
|1966-07-15|     Brazil|    Hungary|           1|            3|FIFA World Cup|        Liverpool|     England|
|1966-07-1

In [87]:
df_fwc_brasil.createOrReplaceTempView('v_brasil_fwc')

In [89]:
spark.sql('select * from v_brasil_fwc where GOL_MANDANTE = GOL_VISITANTE').show()

+----------+---------+--------------+------------+-------------+--------------+-----------------+-------------+
|      DATA| MANDANTE|     VISITANTE|GOL_MANDANTE|GOL_VISITANTE|       TORNEIO|           CIDADE|         PAIS|
+----------+---------+--------------+------------+-------------+--------------+-----------------+-------------+
|1938-06-12|   Brazil|Czechoslovakia|           1|            1|FIFA World Cup|         Bordeaux|       France|
|1950-06-28|   Brazil|   Switzerland|           2|            2|FIFA World Cup|        São Paulo|       Brazil|
|1954-06-19|   Brazil|    Yugoslavia|           1|            1|FIFA World Cup|         Lausanne|  Switzerland|
|1958-06-11|   Brazil|       England|           0|            0|FIFA World Cup|       Gothenburg|       Sweden|
|1962-06-02|   Brazil|Czechoslovakia|           0|            0|FIFA World Cup|     Viña del Mar|        Chile|
|1974-06-13|   Brazil|    Yugoslavia|           0|            0|FIFA World Cup|Frankfurt am Main|      G

In [100]:
spark.sql(" \
    SELECT * \
    FROM v_brasil_fwc \
    WHERE  \
        (GOL_MANDANTE > GOL_VISITANTE AND MANDANTE = 'Brazil')  \
        OR \
        (GOL_MANDANTE < GOL_VISITANTE AND VISITANTE = 'Brazil')\
").count()


73

In [96]:
spark.sql('select * from v_brasil_fwc where (GOL_MANDANTE < GOL_VISITANTE and MANDANTE = "Brazil") or (GOL_MANDANTE > GOL_VISITANTE and VISITANTE = "Brazil") ').count()

18